In [ ]:
# ==============================================================================
# Importaciones y Configuración Inicial
# ==============================================================================
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from IPython.display import display
import numpy as np
from sklearn.linear_model import LinearRegression
import os
from datetime import datetime

# ==============================================================================
# SECCIÓN 0: CONSTANTES GLOBALES
# ==============================================================================
# Parámetros de Simulación
NUM_SIMULACIONES = 1000
HORIZONTE_ANIOS = 1
NUM_PASOS = 12
RANDOM_SEED = 42

# Constantes de Cálculo
DT_SIM = HORIZONTE_ANIOS / NUM_PASOS
DT_CALIB = 1.0 / 252.0
BPS_TO_DECIMAL = 10000.0
DAYS_IN_YEAR = 365.0

1) Primero se importan librerías relevantes para el código, tales como:
   - pandas: Se usa para trabajar con tablas de datos, como leer los archivos de Excel.
   - numpy: Es una calculadora científica para todas las operaciones matemáticas complejas.
   - sklearn: Se para una tarea específica, calibrar nuestros modelos de tasas de interés.
   - Otras librerías que sirven para tareas auxiliares, como mostrar ventanas para seleccionar archivos (tkinter) o manejar fechas (datetime).

2) En la sección 0 se definen parámetros y suposiciones del modelo, en esta parte se deben ajustar si es que se quieren tener más simulaciones, un horizonte de años más grande, más números de pasos, etc.
   - NUM_SIMULACIONES = 1000: Se van a generar 1,000 "futuros posibles" o escenarios para medir el riesgo. A más simulaciones, más preciso el resultado, pero más lento el cálculo
   - HORIZONTE_ANIOS = 1: Se mira el riesgo a lo largo de un año.
   - NUM_PASOS = 12: Se dividirá ese año en 12 pasos, es decir, se hará una simulación mensual.
   - RANDOM_SEED = 42: Es un número que asegura que cada vez que se corra el modelo, se obtengan exactamente los mismos resultados.
   - Las Constantes de Cálculo son simplemente valores técnicos que se derivan de los parámetros anteriores (como el tamaño de cada paso de tiempo) o son convenciones de mercado (como el número de días en un año).

In [ ]:
# ==============================================================================
# Funciones Auxiliares (Corresponden a diferentes pasos)
# ==============================================================================

def cargar_y_validar_forwards(ruta_archivo):
    """(Corresponde al Paso 2) Carga y valida el archivo de forwards."""
    if not ruta_archivo:
        raise ValueError("No se seleccionó ningún archivo de forwards.")

    columnas_esperadas = [
        'ID', 'Contraparte', 'Inicio', 'Fin', 'Flujo_Activo', 'Moneda_Flujo_Activo',
        'Strike', 'Flujo_Pasivo', 'Moneda_Flujo_Pasivo', 'Sentido',
        'CDS_Contraparte', 'CDS_Propio', 'Recovery'
    ]
    df = pd.read_excel(ruta_archivo)
    faltantes = set(columnas_esperadas) - set(df.columns)
    if faltantes:
        raise ValueError(f"Faltan columnas en el archivo de forwards: {faltantes}")

    df['Inicio'] = pd.to_datetime(df['Inicio'])
    df['Fin'] = pd.to_datetime(df['Fin'])
    df['ID'] = df['ID'].astype(str)
    return df

--> Esta función es el punto de entrada de los datos del portafolio de forwards, su objetivo es leer un archivo Excel que contiene la lista de contratos de forwards y prepararlos para el análisis, asegurando que los datos sean completos y tengan el formato correcto.

--> Código:

1) Primero, se lee archivo Excel cargado.

2) Segundo, el código define una lista de todas las columnas que son indispensables para los cálculos posteriores. Luego, compara esta lista con las columnas realmente presentes en el archivo Excel. Es necesario que el Excel tenga cargadas las siguientes columnas:

   - ID: Número identificador de cada operación.
   - Contraparte: Contraparte del contrato forward.
   - Inicio: Fecha de inicio del contrato.
   - Fin: Fecha de término del contrato.
   - Flujo_Activo: Nocional que recibo en su moneda original.
   - Moneda_Flujo_Activo: Moneda del nocional activo.
   - Strike: Precio pactado del activo subyacente.
   - Flujo_Pasivo: Nocional que recibo en su moneda original.
   - Sentido: Porsición del contrato forward.
   - Flujo_Pasivo: Nocional que pago en su moneda original.
   - Moneda_Flujo_Pasivo: Moneda del nocional pasivo.
   - CDS_Contraparte: Credit Default Swaps por contraparte en bp (Es importante que los cds sean constantes por contraparte).
   - CDS_Propio: Credit Default Swaps propios.
   - Recovery: Tasa de recuperación dado el default (debe estar expresado entre 0 y 1).

3) Tercero, las columnas Inicio y Fin se convierten a un formato de fecha especial (datetime), lo que permite realizar cálculos con ellas, como determinar los días hasta el vencimiento. La columna ID se convierte a texto (string), para evitar que se interprete como un número y pierda ceros iniciales o se use en cálculos matemáticos indebidos.

4) La función finaliza y devuelve el DataFrame, ahora limpio, validado y con los formatos correctos, listo para ser utilizado por el resto del modelo.

In [ ]:
def cargar_y_organizar_insumos(ruta_archivo, monedas_unicas, moneda_valoracion, t0):
    """(Corresponde a los Pasos 4, 5, 6) Carga y procesa todas las hojas del archivo de insumos."""
    if not ruta_archivo:
        raise ValueError("No se seleccionó archivo de insumos.")
        
    xls = pd.ExcelFile(ruta_archivo)
    
    curvas_descuento = {}
    tasas_historicas = {}
    datos_fx = {}
    
    print("\n--- Cargando insumos de mercado ---")
    for moneda in monedas_unicas:
        try:
            sheet_name = f"Desc_{moneda}"
            df_curva = pd.read_excel(xls, sheet_name=sheet_name)
            curvas_descuento[moneda] = df_curva
            print(f"  ✅ Curva de descuento '{sheet_name}' cargada.")
        except Exception as e:
            raise ValueError(f"No se pudo leer la hoja de curva para {moneda}. Error: {e}")

        try:
            sheet_name = f"Tasas_{moneda}"
            df_tasa = pd.read_excel(xls, sheet_name=sheet_name)
            df_tasa['Date'] = pd.to_datetime(df_tasa['Date'], dayfirst=True)
            df_tasa = df_tasa.sort_values('Date')
            tasas_historicas[moneda] = df_tasa.set_index('Date')['Rate'] / 100.0
            print(f"  ✅ Tasas históricas '{sheet_name}' cargadas y ordenadas.")
        except Exception as e:
            raise ValueError(f"No se pudo leer la hoja de tasas históricas para {moneda}. Error: {e}")

    monedas_relacionadas = [m for m in monedas_unicas if m != moneda_valoracion]
    for moneda_fx in monedas_relacionadas:
        par_directo = f"{moneda_fx}{moneda_valoracion}"
        par_inverso = f"{moneda_valoracion}{moneda_fx}"
        
        try:
            sheet_name = f"TC_{par_directo}"
            df_fx = pd.read_excel(xls, sheet_name=sheet_name)
            print(f"  ✅ Tipo de cambio '{sheet_name}' cargado (directo).")
        except ValueError:
            try:
                sheet_name = f"TC_{par_inverso}"
                df_fx = pd.read_excel(xls, sheet_name=sheet_name)
                print(f"  ✅ Tipo de cambio '{sheet_name}' cargado (inverso). Se invertirán los precios.")
                df_fx['Price'] = 1.0 / df_fx['Price']
            except ValueError:
                raise ValueError(f"No se pudo encontrar la hoja de TC ni como 'TC_{par_directo}' ni como 'TC_{par_inverso}'.")

        df_fx['Date'] = pd.to_datetime(df_fx['Date'], dayfirst=True)
        df_fx = df_fx.sort_values('Date')
        datos_fx[moneda_fx] = df_fx.set_index('Date')['Price']

    print("✅ Insumos de mercado cargados y organizados.")
    return curvas_descuento, tasas_historicas, datos_fx, monedas_relacionadas

--> Esta función es la responsable de cargar todos los datos de mercado necesarios para valorar los forwards y simular los factores de riesgo. Centraliza la lectura de un único archivo Excel que contiene múltiples hojas, cada una con un insumo financiero específico (curvas de interés, tasas históricas y tipos de cambio).

--> Código:

1) Primero, se abre el archivo Excel de manera eficiente usando pd.ExcelFile. Esto permite leer múltiples hojas sin tener que reabrir el archivo cada vez. Luego, se crean "diccionarios" vacíos. Un diccionario es una estructura de datos que almacenará la información de mercado asociando cada dato a su moneda correspondiente (ej. la curva de USD se guardará con la clave "USD").

2) Segundo, el código itera sobre cada moneda identificada en el portafolio, para cada moneda: 
    - Busca una hoja con el nombre Desc_MONEDA (ej. Desc_USD). Esta hoja contiene la curva de tipos cero o "spot" para esa moneda en la fecha de valoración. Es la que se utiliza para calcular el valor presente de un flujo de caja futuro, se usa para la valoración en t=0 y para descontar el CVA/DVA a valor de hoy.
    - Busca una hoja llamada Tasas_MONEDA (ej. Tasas_CLP). Contiene la serie de tiempo de la tasa corta de esa moneda. Los datos históricos de tasas son necesarios para calibrar el modelo estocástico (Vasicek en este caso). La calibración consiste en encontrar los parámetros del modelo (a, b, sigma) que mejor se ajustan al comportamiento observado de las tasas en el pasado. Se asume que esta dinámica persistirá en el futuro.

3) Para cada moneda extranjera, intenta encontrar la hoja del tipo de cambio en dos formatos posibles.
    - Primero, busca el par MonedaExtranjera/MonedaBase (ej. TC_CLPUSD).
    - Si no lo encuentra, busca el par inverso MonedaBase/MonedaExtranjera (ej. TC_USDCLP). Si lo halla, carga los datos y los invierte matemáticamente (1.0 / Precio) para estandarizarlos.

4) La función finaliza entregando tres diccionarios que contienen todos los datos de mercado, limpios y organizados por moneda, listos para ser usados en la calibración y simulación. 

In [ ]:
def calibrar_vasicek(tasas_historicas):
    """Calibra el modelo Vasicek para cada moneda."""
    parametros_vasicek = {}
    print("\n--- Calibrando modelos Vasicek ---")
    for moneda, serie_tasas in tasas_historicas.items():
        #(1)
        r = serie_tasas.dropna().values
        X, y = r[:-1].reshape(-1, 1), r[1:].reshape(-1, 1)
        #(2)
        modelo = LinearRegression().fit(X, y)
        
        beta = modelo.coef_[0, 0]
        alpha = modelo.intercept_[0]
        #(3)
        if beta <= 0:
            print(f"ADVERTENCIA: Beta para {moneda} fue {beta:.4f}, no es positivo. Se ajustará para continuar.")
            beta = 1e-6
        
        a = -np.log(beta) / DT_CALIB
        b = alpha / (1.0 - beta)
        #(4)
        residuales = (y - modelo.predict(X)).flatten()
        sigma = np.std(residuales, ddof=0) / np.sqrt(DT_CALIB)
        #(5)
        parametros_vasicek[moneda] = {'a': a, 'b': b, 'sigma': sigma}
        print(f"  {moneda}: a={a:.4f}, b={b:.4f}, sigma={sigma:.4f}")
        
    return parametros_vasicek

--> El propósito de esta función es encontrar los parámetros del modelo de Vasicek que mejor describen el comportamiento histórico de las tasas de interés para cada moneda. En otras palabras, "enseña" al modelo cómo se han movido las tasas en el pasado para que pueda simular movimientos realistas en el futuro.

--> Teoría: 

° El modelo de Vasicek postula que la tasa de interés corta (r) sigue un proceso estocástico descrito por la siguiente Ecuación Diferencial Estocástica (SDE):
dr(t) = a(b - r(t))dt + σdW(t)

Donde:

- r(t): Es la tasa de interés corta en el tiempo t.
- a: Es la velocidad de reversión a la media. Un a alto significa que la tasa vuelve rápidamente a su nivel de largo plazo.
- b: Es el nivel de largo plazo o la media a la que la tasa tiende a revertir.
- σ (sigma): Es la volatilidad instantánea de la tasa.
- dW(t): Es un término aleatorio (un proceso de Wiener) que representa el "ruido" o los shocks del mercado.

° Para poder usar este modelo en un computador, se necesita una versión en tiempo discreto. Una aproximación común (el esquema de Euler) es:

r(t+Δt) ≈ r(t) + a(b - r(t))Δt + σ√Δt * Z (Donde Z es una variable aleatoria con distribución normal estándar).

° Reordenando esta ecuación, se puede expresar r(t+Δt) como una función lineal de r(t) más un término de error:

r(t+Δt) ≈ (abΔt) + (1 - a*Δt)r(t) + (σ√Δt * Z)

° Esta forma es una regresión lineal: y = intercepto + pendiente * x + error, 

Donde:

- y = r(t+Δt) (la tasa de mañana)
- x = r(t) (la tasa de hoy)
- intercepto (α) = a * b * Δt
- pendiente (β) = 1 - a * Δt
- error tiene una desviación estándar relacionada con σ.

--> Código:

1) Se toman las series de tiempo de las tasas de interés. Se crean dos vectores: X contiene las tasas de cada día (ej. r_hoy), y y contiene las tasas del día siguiente (ej. r_mañana). Esto prepara los datos para predecir la tasa de mañana basándose en la de hoy.

2) Se utiliza la librería scikit-learn para ajustar un modelo de regresión lineal a los datos X e y. El modelo encuentra el mejor alpha (intercepto) y beta (pendiente) que describen la relación r_mañana ≈ alpha + beta * r_hoy.

3) Usando las relaciones teóricas α = a*b*Δt y β = 1-a*Δt, y resolviendo para a y b, se obtienen estas fórmulas. El código "traduce" los resultados de la regresión (alpha y beta) a los parámetros del modelo financiero (a y b).DT_CALIB es el Δt de los datos históricos (diario, 1/252).

También se incluye una comprobación de que beta sea positivo, ya que un beta negativo o cero no tendría sentido financiero (implicaría una reversión a la media infinitamente rápida o inexistente) y causaría un error matemático en el logaritmo.

4) Los "residuales" son los errores de predicción del modelo de regresión (la diferencia entre el r_mañana real y el predicho). La teoría dice que la desviación estándar de estos residuales es igual a σ * √Δt.

Por lo tanto, para encontrar σ, se calcula la desviación estándar de los residuales y se divide por la raíz cuadrada de DT_CALIB.

5) Los tres parámetros calculados (a, b, sigma) se guardan en un diccionario asociado a su moneda y la función devuelve este diccionario completo, que ahora contiene los parámetros calibrados para todas las monedas del portafolio.

In [ ]:
def calibrar_y_simular_riesgos(tasas_historicas, datos_fx, monedas_unicas, monedas_relacionadas, moneda_valoracion):
    """ Calibra, construye Cholesky y simula."""
    #(1)
    params_vasicek = calibrar_vasicek(tasas_historicas)
    #(2)
    shocks_list = []
    nombres_vars_tasas = [f"d{m}" for m in monedas_unicas]
    nombres_vars_fx = [f"logRet_{m}{moneda_valoracion}" for m in monedas_relacionadas]
    nombres_vars = nombres_vars_tasas + nombres_vars_fx
    
    for moneda in monedas_unicas:
        shocks_list.append(tasas_historicas[moneda].diff().dropna().rename(f"d{moneda}"))
    for moneda_fx in monedas_relacionadas:
        shocks_list.append(np.log(datos_fx[moneda_fx]).diff().dropna().rename(f"logRet_{moneda_fx}{moneda_valoracion}"))
    #(3)    
    df_shocks = pd.concat(shocks_list, axis=1).dropna()
    cov_matrix = df_shocks.cov().reindex(index=nombres_vars, columns=nombres_vars)
    #(4)
    cholesky_matrix = np.linalg.cholesky(cov_matrix.values)
    print("\n🧮 Matriz de Cholesky calculada.")
    #(5)
    np.random.seed(RANDOM_SEED)
    num_vars = len(nombres_vars)
    #(6)
    epsilon = np.random.standard_normal(size=(NUM_SIMULACIONES, NUM_PASOS, num_vars))
    shocks_corr = np.einsum('ijk,lk->ijl', epsilon, cholesky_matrix)
    #(7)
    simulaciones_tasas = {m: np.zeros((NUM_SIMULACIONES, NUM_PASOS + 1)) for m in monedas_unicas}
    for i, moneda in enumerate(monedas_unicas):
        tasa_inicial = tasas_historicas[moneda].iloc[-1]
        a, b, sigma = params_vasicek[moneda].values()
        simulaciones_tasas[moneda][:, 0] = tasa_inicial
        for t in range(1, NUM_PASOS + 1):
            r_prev = simulaciones_tasas[moneda][:, t - 1]
            Z = shocks_corr[:, t - 1, i]
            r_t = r_prev + a * (b - r_prev) * DT_SIM + sigma * np.sqrt(DT_SIM) * Z
            simulaciones_tasas[moneda][:, t] = r_t
    #(8)        
    simulaciones_tc = {m: np.zeros((NUM_SIMULACIONES, NUM_PASOS + 1)) for m in monedas_relacionadas}
    for i, moneda_fx in enumerate(monedas_relacionadas):
        spot_inicial = datos_fx[moneda_fx].iloc[-1]
        simulaciones_tc[moneda_fx][:, 0] = spot_inicial
        idx_shock_fx = len(monedas_unicas) + i
        r_ext = simulaciones_tasas[moneda_fx]
        r_loc = simulaciones_tasas[moneda_valoracion]
        for t in range(1, NUM_PASOS + 1):
            drift = np.exp((r_ext[:, t-1] - r_loc[:, t-1]) * DT_SIM)
            Z = shocks_corr[:, t-1, idx_shock_fx]
            difusion = np.exp(-0.5 * cov_matrix.iloc[idx_shock_fx, idx_shock_fx] * DT_SIM + np.sqrt(DT_SIM) * Z)
            S_prev = simulaciones_tc[moneda_fx][:, t - 1]
            simulaciones_tc[moneda_fx][:, t] = S_prev * drift * difusion
    #(9)        
    print("✅ Modelos calibrados y factores de riesgo simulados.")
    return simulaciones_tasas, simulaciones_tc

--> El objetivo de esta función es generar las trayectorias futuras para todos los factores de riesgo del mercado (tasas de interés y tipos de cambio). Para lograrlo, realiza lo siguiente: 

- Calibra los modelos de tasas de interés (llamando a la función calibrar_vasicek).
- Mide la correlación histórica entre todos los factores de riesgo.
- Simula los escenarios futuros, asegurando que los movimientos aleatorios respeten la estructura de correlación observada en el mercado.

--> Teoría: 

° En los mercados financieros, los activos no se mueven de forma independiente. Por ejemplo, es común que las tasas de interés de Chile y EE.UU. suban o bajen juntas, o que una subida en la tasa de interés de CLP fortalezca el tipo de cambio CLP/USD. Ignorar estas correlaciones produciría escenarios futuros poco realistas.

° El método estándar para incorporar la correlación en una simulación de Monte Carlo es la Descomposición de Cholesky. Para aplicar Cholesky se deben seguir los siguientes pasos:

- Primero, se calcula la Matriz de Covarianza (Σ), la que mide cómo han variado conjuntamente los cambios diarios (shocks) de todos los factores de riesgo en el pasado.

- Luego, se aplica la descomposición de Cholesky, esta técnica descompone la matriz de covarianza Σ en el producto de una matriz triangular inferior L y su transpuesta Lᵀ (es decir, Σ = L * Lᵀ). La matriz L actúa como una especie de "raíz cuadrada de la matriz de covarianza".

- Después, se tienen generan shocks aleatorios independientes (ε, extraídos de una distribución normal). Al multiplicar estos shocks independientes por la matriz L (shocks_correlacionados = L * ε), se obtienen nuevos shocks que tienen exactamente la misma estructura de covarianza y correlación que los datos históricos originales.

1) Se invoca la función anterior para obtener los parámetros a, b y sigma para cada moneda.

2) El objetivo de este segmento es transformar las series de tiempo de precios y tasas (que son niveles, como 950 CLP/USD o una tasa del 5%) en series de tiempo de cambios diarios o "shocks".

- shocks_list = []: Se crea una lista vacía que servirá para almacenar las series de tiempo de los shocks de cada factor de riesgo.
- nombres_vars...: Se crean listas con los nombres que se le darán a cada serie de shock. Por ejemplo, si las monedas son "CLP" y "USD" (valoración), los nombres serán ['dCLP', 'dUSD', 'logRet_CLPUSD'].
- Se itera sobre cada moneda (ej. "CLP", "USD"). 
- tasas_historicas[moneda]: Se toma la serie de tiempo de la tasa de interés para esa moneda.
- .diff(): Calcula la diferencia entre cada día y el día anterior. Si la serie era [5.0, 5.1, 5.05], el resultado de .diff() sería [NaN, 0.1, -0.05]. Este es el "shock" o cambio diario de la tasa. 
- .dropna(): El primer elemento del resultado de .diff() es siempre NaN (No es un Número) porque no tiene un día anterior con el cual compararse. .dropna() elimina este valor nulo.
- .rename(f"d{moneda}"): Le da un nombre a la nueva serie de shocks (ej. "dCLP"), que coincide con los nombres definidos en el Paso 1.
shocks_list.append(...): Se añade la serie de shocks de esta moneda a la lista general.
- np.log(datos_fx[moneda_fx]): Primero, se aplica el logaritmo natural a la serie de precios del tipo de cambio.
- .diff(): Luego, se calcula la diferencia de los logaritmos.

3) Se calculan los cambios diarios ("shocks") para cada factor de riesgo:

- Para las tasas, se usa la diferencia simple: tasa_hoy - tasa_ayer.

- Para el tipo de cambio, se usan los retornos logarítmicos: log(precio_hoy / precio_ayer). Esta es la práctica estándar, ya que los log-retornos tienen mejores propiedades estadísticas.

- Luego, se calcula la matriz de covarianza de estos shocks históricos.

4) Se utiliza la función de álgebra lineal de NumPy para obtener la matriz L (llamada cholesky_matrix) a partir de la matriz de covarianza.

Después, se cuentan cuántos factores de riesgo totales se están modelando y guardar ese número en una variable llamada num_vars.

5) Para la parte aleatoria, se utiliza np.random.seed(RANDOM_SEED), con Random_SEED definida previamente en los aspectos generales.

6) Se generan NUM_SIMULACIONES x NUM_PASOS shocks aleatorios independientes para cada factor de riesgo (epsilon).

Se multiplican estos shocks independientes por la cholesky_matrix. El resultado (shocks_corr) es un conjunto de shocks aleatorios que ahora están correlacionados de manera más realista.

7) Usando la ecuación de discretización de Vasicek, se simulan las NUM_SIMULACIONES trayectorias para la tasa de cada moneda. El término aleatorio Z para cada moneda y en cada paso de tiempo se toma directamente de la matriz de shocks correlacionados (shocks_corr) generada en el paso anterior.

8) Se simula el tipo de cambio usando la teoría de la paridad de tasas de interés, que es el estándar para la valoración de derivados de FX.

- drift (Término de Tendencia): Representa la tendencia del tipo de cambio, que depende del diferencial entre la tasa de interés extranjera (r_ext) y la local (r_loc). Si la tasa extranjera es mayor, el tipo de cambio tiende a depreciarse para evitar arbitrajes.

- difusion (Término de Difusión): Representa el movimiento aleatorio del tipo de cambio. El término Z se toma de la columna correspondiente de la matriz de shocks correlacionados.

9) La función devuelve dos diccionarios: uno con todas las trayectorias simuladas para las tasas de interés y otro con las trayectorias para los tipos de cambio. Estos datos son la materia prima para la valoración del portafolio en el siguiente paso.

In [ ]:
def valorar_portafolio_vectorizado(forwards_df, t0, moneda_valoracion, curvas_descuento, sim_tasas, sim_tc):
    """
    (Pasos 11, 12, 13) Valora el portafolio usando las curvas de tipos simuladas.
    """
    print("\n--- Valorando portafolio (usando curvas de descuento simuladas) ---")
    
    # 1. Simulación de Curvas de Descuento (Shift Paralelo)
    dfs_simulados_array = {}
    for moneda, curva_df in curvas_descuento.items():
        dias_tenor = curva_df['Days'].values
        curva0 = curva_df['Zero Rate'].values / 100.0
        r_short0 = sim_tasas[moneda][:, 0][0]
        
        df_array = np.zeros((NUM_SIMULACIONES, NUM_PASOS + 1, len(dias_tenor)))
        df_array[:, 0, :] = np.exp(-curva0 * dias_tenor / DAYS_IN_YEAR)
        
        delta_r = sim_tasas[moneda][:, 1:] - r_short0
        curvas_sim = curva0 + delta_r[:, :, np.newaxis]
        df_array[:, 1:, :] = np.exp(-curvas_sim * dias_tenor[np.newaxis, np.newaxis, :] / DAYS_IN_YEAR)
        
        dfs_simulados_array[moneda] = df_array

    # 2. Preparación para la Valoración
    num_forwards = len(forwards_df)
    valores_mtm = np.zeros((num_forwards, NUM_SIMULACIONES, NUM_PASOS + 1))
    
    dias_vencimiento = (forwards_df['Fin'] - t0).dt.days.values
    tiempos_sim_dias = np.linspace(0, HORIZONTE_ANIOS * DAYS_IN_YEAR, NUM_PASOS + 1)
    dias_rem_grid = np.maximum(0, dias_vencimiento[:, np.newaxis] - tiempos_sim_dias)

    # 3. Valoración por Forward
    print("  Interpolando factores de descuento y valorando forwards...")
    for i, fwd in forwards_df.iterrows():
        # Flujo Activo
        moneda_a = fwd['Moneda_Flujo_Activo']
        dias_tenor_a = curvas_descuento[moneda_a]['Days'].values
        sim_dfs_a = dfs_simulados_array[moneda_a]
        
        dfs_interpolados_a = np.zeros((NUM_SIMULACIONES, NUM_PASOS + 1))
        for t in range(NUM_PASOS + 1):
            d_rem = dias_rem_grid[i, t]
            dfs_interpolados_a[:, t] = np.array([np.interp(d_rem, dias_tenor_a, sim_dfs_a[s, t, :]) for s in range(NUM_SIMULACIONES)])

        fx_a = np.ones((NUM_SIMULACIONES, NUM_PASOS + 1))
        if moneda_a != moneda_valoracion:
            fx_a = sim_tc[moneda_a]
        pv_activo = fwd['Flujo_Activo'] * dfs_interpolados_a * fx_a

        # Flujo Pasivo
        moneda_p = fwd['Moneda_Flujo_Pasivo']
        dias_tenor_p = curvas_descuento[moneda_p]['Days'].values
        sim_dfs_p = dfs_simulados_array[moneda_p]
        
        dfs_interpolados_p = np.zeros((NUM_SIMULACIONES, NUM_PASOS + 1))
        for t in range(NUM_PASOS + 1):
            d_rem = dias_rem_grid[i, t]
            dfs_interpolados_p[:, t] = np.array([np.interp(d_rem, dias_tenor_p, sim_dfs_p[s, t, :]) for s in range(NUM_SIMULACIONES)])

        fx_p = np.ones((NUM_SIMULACIONES, NUM_PASOS + 1))
        if moneda_p != moneda_valoracion:
            fx_p = sim_tc[moneda_p]
        pv_pasivo = fwd['Flujo_Pasivo'] * dfs_interpolados_p * fx_p
        
        valores_mtm[i, :, :] = pv_activo - pv_pasivo

    print("✅ Portafolio valorado en todas las simulaciones y pasos de tiempo.")
    return valores_mtm, dfs_simulados_array

--> El objetivo de esta función es calcular el valor de mercado (Mark-to-Market, MtM) de cada forward del portafolio, en cada uno de los NUM_PASOS de tiempo y para cada una de las NUM_SIMULACIONES. El resultado final es un gran "cubo" de datos de valoraciones que será la base para el cálculo del CVA y DVA.

--> Teoría: 

° La valoración de un forward en cualquier momento del tiempo t se basa en la fórmula fundamental del valor presente: MtM(t) = VP(Flujo_Activo, t) - VP(Flujo_Pasivo, t)

Donde:

- VP(Flujo, t) es el Valor Presente de un flujo de caja futuro, visto desde el tiempo t.
- VP(Flujo, t) = Flujo_Nominal * Factor_Descuento(t, T_venc) * Tipo_Cambio(t)

° Dado que el Factor de Descuento como el Tipo de Cambio son estocásticos y cambian en cada simulación y en cada paso de tiempo, es necesario realizar lo siguiente: 

  - Simulación de Curvas (Shift Paralelo): Para obtener los factores de descuento futuros, el modelo utiliza una suposición simplificadora pero común: la curva de tasas de interés se moverá en "shifts paralelos". Esto significa que si la tasa corta simulada r(t) sube un 0.1% respecto a su valor inicial, se asume que toda la curva de tasas sube un 0.1%.

  - Fórmula: Curva_Simulada(t) = Curva_Inicial + (r(t) - r_inicial). A partir de esta curva simulada, se pueden calcular los factores de descuento para cualquier vencimiento.

  - Las curvas de descuento del mercado se definen solo para ciertos plazos estándar (ej. 30, 60, 90, 360 días). Sin embargo, un forward puede vencer en un plazo intermedio (ej. 85 días). La interpolación lineal es una técnica matemática para estimar el factor de descuento para ese plazo intermedio, basándose en los dos plazos estándar más cercanos.

--> Código: 

1) Este bloque implementa la teoría del "shift paralelo".

- delta_r: Calcula la diferencia entre la tasa corta simulada en cada paso de tiempo y la tasa corta inicial. Este es el "shift".
- curvas_sim: Suma este "shift" a la curva de tasas inicial para obtener la curva de tasas simulada en cada escenario.
- np.exp(...): Usa la fórmula exp(-tasa * tiempo) para convertir las curvas de tasas simuladas en factores de descuento simulados.

El resultado, dfs_simulados_array, es un diccionario que contiene, para cada moneda, un "cubo" de factores de descuento para todos los tenores de la curva, en cada simulación y en cada paso de tiempo.

2) Se preparan las estructuras de datos.

- valores_mtm: Se crea un "cubo" vacío con las dimensiones correctas (nº de forwards x nº de simulaciones x nº de pasos) que se llenará con los resultados de la valoración.
- dias_rem_grid: Se calcula una matriz que contiene, para cada forward y cada paso de tiempo de la simulación, cuántos días quedan hasta su vencimiento.

3) Este es el bucle principal de valoración. Itera sobre cada forward y realiza la valoración completa.

- Obtener Datos: Para cada forward, se identifican las monedas de sus flujos y se extraen los datos de simulación correspondientes (curvas de descuento y tipos de cambio).

- Interpolar: Usando la función np.interp, se obtiene el factor de descuento preciso para el vencimiento del forward. Se toma la curva de descuento simulada para una simulación s y un tiempo t, y se interpola para encontrar el valor en los dias_rem_grid correspondientes.

- Calcular Valor Presente (VP): Se calcula el VP de la pata activa y de la pata pasiva usando la fórmula VP = Flujo_Nominal * Factor_Descuento_Interpolado * Tipo_Cambio_Simulado.

- Calcular MtM: Se restan los VPs (pv_activo - pv_pasivo) para obtener el valor del forward en esa simulación y en ese paso de tiempo.

- Almacenar Resultado: El MtM calculado se almacena en el "cubo" valores_mtm.

4) La función devuelve el cubo valores_mtm completo, que contiene todas las valoraciones, y también dfs_simulados_array, que contiene las curvas simuladas y se usará para el reporte.

In [ ]:
def calcular_xva_con_neteo(forwards_df, valores_mtm, curvas_descuento, moneda_valoracion):
    """ Calcula EE, CVA, DVA, y BVA con neteo.
    """
    print("\n--- Calculando CVA, DVA y BVA con neteo ---")

    #(1)
    lgd = 1.0 - forwards_df['Recovery'].values
    lambda_cparty = (forwards_df['CDS_Contraparte'].values / BPS_TO_DECIMAL) / lgd
    lambda_propio = (forwards_df['CDS_Propio'].values / BPS_TO_DECIMAL) / lgd
    #(2)
    tiempos = np.linspace(0, HORIZONTE_ANIOS, NUM_PASOS + 1)
    
    curva_descuento_valoracion = curvas_descuento[moneda_valoracion]
    tasas_descuento_spot = np.interp(
        tiempos, 
        curva_descuento_valoracion['Days'].values / DAYS_IN_YEAR, 
        curva_descuento_valoracion['Zero Rate'].values / 100.0
    )
    factores_descuento = np.exp(-tasas_descuento_spot * tiempos)
    print(f"  Factores de descuento para CVA/DVA basados en la curva '{moneda_valoracion}'.")
    #(3)
    pd_acum_cparty = 1 - np.exp(-lambda_cparty[:, np.newaxis] * tiempos)
    pd_marg_cparty = np.diff(pd_acum_cparty, axis=1, prepend=0)
    
    pd_acum_propio = 1 - np.exp(-lambda_propio[:, np.newaxis] * tiempos)
    pd_marg_propio = np.diff(pd_acum_propio, axis=1, prepend=0)
    #(4)
    ee_standalone = np.maximum(valores_mtm, 0).mean(axis=1)
    ene_standalone = np.minimum(valores_mtm, 0).mean(axis=1)
    
    df_results = forwards_df[['ID', 'Contraparte']].copy()
    df_results['CVA'], df_results['DVA'] = 0.0, 0.0
    #(5)
    for cparty, group in forwards_df.groupby('Contraparte'):
        idx = group.index
        mtm_cparty = valores_mtm[idx, :, :].sum(axis=0)
        ee_cparty = np.maximum(mtm_cparty, 0).mean(axis=0)
        ene_cparty = np.minimum(mtm_cparty, 0).mean(axis=0)
        
        cva_total_cparty = (ee_cparty * pd_marg_cparty[idx[0], :] * lgd[idx[0]] * factores_descuento).sum()
        dva_total_cparty = (-ene_cparty * pd_marg_propio[idx[0], :] * lgd[idx[0]] * factores_descuento).sum()

        ee_standalone_cparty_sum = ee_standalone[idx, :].sum(axis=1)
        ene_standalone_cparty_sum = ene_standalone[idx, :].sum(axis=1)
        
        total_ee = ee_standalone_cparty_sum.sum()
        cva_prorrateado = cva_total_cparty * (ee_standalone_cparty_sum / total_ee) if total_ee > 0 else 0
        
        total_ene = ene_standalone_cparty_sum.sum()
        dva_prorrateado = dva_total_cparty * (ene_standalone_cparty_sum / total_ene) if total_ene != 0 else 0

        df_results.loc[idx, 'CVA'] = cva_prorrateado
        df_results.loc[idx, 'DVA'] = dva_prorrateado
    #(6)
    df_results['BVA'] = df_results['CVA'] - df_results['DVA']
    
    ee_total_agregado = np.maximum(valores_mtm.sum(axis=0), 0).mean(axis=0)
    ee_df = pd.DataFrame({'Tiempo (años)': tiempos, 'EE_Total': ee_total_agregado})
    
    print("✅ CVA, DVA, y BVA calculados.")
    return df_results.set_index('ID'), ee_df

--> El objetivo de esta función es calcular los principales ajustes de valoración por riesgo de crédito: CVA, DVA y BVA. Utiliza el "cubo" de valores MtM simulados para derivar las exposiciones futuras y las combina con las probabilidades de default para cuantificar el riesgo. Una característica clave es que implementa el neteo por contraparte, que es fundamental en la práctica.

--> Teoría: 

° El CVA (Credit Valuation Adjustment) y el DVA (Debit Valuation Adjustment) son esencialmente el valor presente del riesgo de crédito. Se calculan integrando tres componentes a lo largo del tiempo:

CVA = Σ [ EE(t) * PD(t, t+Δt) * LGD ] * DF(0, t)
DVA = Σ [ ENE(t) * PD_propia(t, t+Δt) * LGD_propia ] * DF(0, t)

Donde:

- EE(t) (Exposición Esperada): Es el promedio de los valores de mercado positivos en el tiempo t. Representa cuánto se espera que te deba la contraparte si no hace default.

- ENE(t) (Exposición Esperada Negativa): Es el promedio de los valores de mercado negativos. Representa cuánto se espera que tú le debas a la contraparte.

- PD(t, t+Δt): Es la probabilidad de que la contraparte haga default en un pequeño intervalo de tiempo (t, t+Δt), dado que ha sobrevivido hasta t. Se deriva de los spreads de CDS.

- LGD = 1 - Tasa_Recuperacion: Es el porcentaje de la exposición que se perdería en caso de default.

- DF(0, t): Trae a valor presente el costo futuro del riesgo de crédito, usando la curva libre de riesgo.

- Neteo (Netting): Los acuerdos de neteo permiten que, en caso de default, todas las operaciones con una misma contraparte se liquiden como una única obligación neta. Esto significa que la exposición no es la suma de las exposiciones individuales, sino la exposición del portafolio neto con esa contraparte. Es un mecanismo de mitigación de riesgo crucial.

° No se puede simplemente decir "un spread del 2.5% significa una probabilidad de default del 2.5%". La relación es más sutil y se basa en un concepto clave: la Tasa de Riesgo o Hazard Rate (λ).

- La tasa de riesgo λ es la probabilidad instantánea de default, asumiendo que la entidad ha sobrevivido hasta ese momento. Es análoga a una tasa de interés, pero en lugar de acumular dinero, acumula probabilidad de default.

- La conexión fundamental entre el spread del CDS y λ se basa en un principio de no arbitraje. A grandes rasgos, el valor presente de los pagos que un vendedor de CDS espera recibir (la prima) debe ser igual al valor presente de lo que espera pagar (la pérdida en caso de default).

- Esto lleva a la siguiente aproximación, que es muy utilizada en la industria por su simplicidad y robustez: Spread_CDS ≈ λ * LGD

° Una vez que se tiene λ, es posible modelar la probabilidad de que una empresa no quiebre (es decir, que "sobreviva") hasta un tiempo t. Asumiendo que λ es constante, la fórmula es idéntica a la del decaimiento exponencial (como en la radioactividad o el interés compuesto continuo en reversa):

- Probabilidad de Supervivencia hasta t (PS(t)) = exp(-λt)

- Si t=0, PS(0) = e⁰ = 1 (100% de probabilidad de no haber quebrado aún).

- A medida que t aumenta, la probabilidad de supervivencia disminuye exponencialmente.

° La probabilidad de que una empresa sí quiebre en algún momento antes de t es simplemente el complemento de la probabilidad de supervivencia: Probabilidad de Default Acumulada hasta t (PD_Acum(t)) = 1 - PS(t) = 1 - exp(-λt).

° Para el cálculo del CVA, no sirve tener la probabilidad acumulada, sino la probabilidad de que el default ocurra específicamente en un intervalo de tiempo, por ejemplo, entre el mes 6 y el mes 7. Esta es la Probabilidad de Default Marginal.

- Se calcula de la siguiente forma, es la diferencia en la probabilidad acumulada entre el final y el principio del intervalo:

- Probabilidad Marginal para el intervalo (t₁, t₂) = PD_Acum(t₂) - PD_Acum(t₁)


--> Código: 

1) Se calculan los componentes básicos del riesgo de crédito a partir de los insumos:

- lgd: Se calcula la Pérdida Dado el Incumplimiento.
- lambda...: Se convierte el spread de CDS (en puntos base) a una tasa de riesgo instantánea (hazard rate, λ), que es el parámetro fundamental para calcular las probabilidades de default.

2) Se obtienen los factores de descuento de la curva libre de riesgo de la moneda de valoración. Estos se usarán al final para traer a valor presente los costos de CVA y DVA calculados en el futuro.

3) pd_acum_cparty: Se calcula la probabilidad de default acumulada hasta cada punto en el tiempo, usando la fórmula 1 - exp(-λ*t).

   pd_marg_cparty: Se calcula la diferencia entre las probabilidades acumuladas en cada paso. El resultado es la probabilidad de default marginal: la probabilidad de que el default ocurra específicamente en ese intervalo de tiempo. Este es el PD(t, t+Δt) de la fórmula teórica.

4) Se calculan los perfiles de exposición para cada forward de forma individual, sin considerar el neteo.

- ee_standalone: Se toman todos los valores de MtM simulados, se reemplazan los negativos por cero (np.maximum(..., 0)), y luego se promedian sobre todas las simulaciones.

- ene_standalone: Se hace lo mismo, pero reemplazando los valores positivos por cero (np.minimum(..., 0)).

5) Este es el bucle que implementa el neteo.

- groupby('Contraparte'): Se agrupan todas las operaciones pertenecientes a la misma contraparte.

- mtm_cparty = ... .sum(axis=0): Se suman los MtM de todas las operaciones de esa contraparte en cada simulación y paso de tiempo. Este es el portafolio neto.

- ee_cparty: Se calcula la Exposición Esperada sobre este portafolio neto.

- cva_total_cparty: Se calcula el CVA total para esa contraparte, usando la EE del portafolio neto.

- Una vez que se tiene el CVA/DVA total del portafolio con la contraparte, este se distribuye o "prorratea" a cada operación individual. La lógica de prorrateo utilizada es en base a la contribución de cada operación a la exposición total. Esto permite asignar un valor de CVA/DVA a nivel de transacción individual.

6) Se calcula el BVA (Bilateral Valuation Adjustment), que es simplemente CVA - DVA. Representa el ajuste de valoración total por riesgo de crédito bilateral. La función devuelve el DataFrame con los resultados finales por operación.

In [ ]:
def generar_reporte_detallado(ruta_salida, forwards_df, df_xva, df_ee, valores_mtm, simulaciones_tc, dfs_simulados, curvas_descuento, moneda_valoracion, t0, ruta_forwards, ruta_insumos, tc_reporte_inverso):
    print("\n--- Generando reporte detallado ---")
    
    # 1. Hoja de Parámetros de Simulación
    params_data = {
        "Parámetro": [
            "Fecha de Valoración", "Moneda de Valoración", "Horizonte (Años)",
            "Número de Simulaciones", "Pasos de Simulación", "Random Seed",
            "Archivo Forwards", "Archivo Insumos", "Formato TC Reporte"
        ],
        "Valor": [
            t0.strftime('%Y-%m-%d'), moneda_valoracion, HORIZONTE_ANIOS,
            NUM_SIMULACIONES, NUM_PASOS, RANDOM_SEED,
            os.path.basename(ruta_forwards), os.path.basename(ruta_insumos),
            f"{moneda_valoracion}/Moneda" if tc_reporte_inverso else f"Moneda/{moneda_valoracion}"
        ]
    }
    df_params = pd.DataFrame(params_data)

    # 2. Valor Forward Esperado
    df_valor_esperado = pd.DataFrame(
        valores_mtm.mean(axis=1).T,
        index=pd.Index(np.linspace(0, HORIZONTE_ANIOS, NUM_PASOS + 1), name="Tiempo (años)"),
        columns=forwards_df['ID']
    )
    
    # 3. Estadísticas de Tipos de Cambio Simulados
    dfs_fx_stats = []
    for moneda_fx, sim_array in simulaciones_tc.items():
        
        ## <-- AJUSTE TC REPORTE: Invierte el TC si el usuario lo solicita
        sim_array_reporte = sim_array
        if tc_reporte_inverso:
            par = f"{moneda_valoracion}/{moneda_fx}"
            # Evitar división por cero si alguna simulación da 0
            with np.errstate(divide='ignore'):
                sim_array_reporte = 1.0 / sim_array
                sim_array_reporte[sim_array == 0] = 0 # Manejar el caso de 0
        else:
            par = f"{moneda_fx}/{moneda_valoracion}"
        
        df = pd.DataFrame({
            f'Media_{par}': sim_array_reporte.mean(axis=0),
            f'Std_{par}': sim_array_reporte.std(axis=0),
            f'Mediana_{par}': np.median(sim_array_reporte, axis=0),
            f'P5_{par}': np.percentile(sim_array_reporte, 5, axis=0),
            f'P95_{par}': np.percentile(sim_array_reporte, 95, axis=0),
        }, index=pd.Index(np.linspace(0, HORIZONTE_ANIOS, NUM_PASOS + 1), name="Tiempo (años)"))
        dfs_fx_stats.append(df)
    df_fx_report = pd.concat(dfs_fx_stats, axis=1) if dfs_fx_stats else pd.DataFrame()

    # 4. Curvas de Descuento y Tasas Promedio
    reportes_curvas = {}
    for moneda, df_sim in dfs_simulados.items():
        dias = curvas_descuento[moneda]['Days'].values
        pasos_cols = [f"T_{t:.2f}a" for t in np.linspace(0, HORIZONTE_ANIOS, NUM_PASOS + 1)]
        
        df_promedio = df_sim.mean(axis=0).T
        df_df_prom = pd.DataFrame(df_promedio, columns=pasos_cols)
        df_df_prom.insert(0, "Days", dias)
        
        with np.errstate(divide='ignore', invalid='ignore'):
            r_equiv = -np.log(df_promedio) * DAYS_IN_YEAR / dias[:, np.newaxis] * 100
        df_r_prom = pd.DataFrame(r_equiv, columns=pasos_cols)
        df_r_prom.insert(0, "Days", dias)
        
        reportes_curvas[f"DF_Prom_{moneda}"] = df_df_prom
        reportes_curvas[f"Tasa_Prom_{moneda}"] = df_r_prom

    # 5. Escribir todo a Excel
    with pd.ExcelWriter(ruta_salida, engine="xlsxwriter") as writer:
        df_params.to_excel(writer, sheet_name="Parametros", index=False)
        df_xva.to_excel(writer, sheet_name="CVA_DVA_BVA", index=True)
        df_ee.to_excel(writer, sheet_name="Exposicion_Esperada_Total", index=False)
        df_valor_esperado.to_excel(writer, sheet_name="Valor_Forward_Esperado", index=True)
        if not df_fx_report.empty:
            df_fx_report.to_excel(writer, sheet_name="FX_Simuladas_Stats", float_format="%.6f")
        for nombre_hoja, df_reporte in reportes_curvas.items():
            df_reporte.to_excel(writer, sheet_name=nombre_hoja[:31], index=False, float_format="%.8f")

    print(f"📁 Reporte detallado guardado exitosamente en: {os.path.abspath(ruta_salida)}")

--> Esta función actúa como el módulo de comunicación final del modelo. Su objetivo es tomar todos los resultados complejos y los datos intermedios generados (CVA, DVA, exposiciones, simulaciones, etc.) y presentarlos de forma estructurada en un archivo Excel con múltiples hojas.

--> Código: 

1) Se crea una tabla simple que resume todos los parámetros y configuraciones utilizados en la ejecución.

- Esta hoja es fundamental para la trazabilidad. Permite que cualquier persona pueda entender exactamente bajo qué supuestos se generaron los resultados y pueda reproducir el cálculo de manera idéntica.

2) Se toma el "cubo" de valoraciones (valores_mtm) y se calcula el promedio sobre todas las simulaciones para cada forward y cada paso de tiempo.

- Esta tabla muestra la trayectoria esperada del valor de mercado de cada contrato a lo largo del tiempo. Permite visualizar si se espera que un contrato gane o pierda valor en el futuro.

3) Para cada tipo de cambio simulado, se calculan varias estadísticas clave en cada paso de tiempo:

- Media: El valor promedio esperado.

- Std (Desviación Estándar): Una medida de la volatilidad o la dispersión de los resultados. Un valor alto indica mayor incertidumbre.

- Mediana y Percentiles (P5, P95): Proporcionan una idea de la distribución de los resultados. El rango entre el percentil 5 y el 95 contiene el 90% central de todos los escenarios simulados, dando una "banda de confianza" de los movimientos futuros.

- Incluye la lógica para invertir el tipo de cambio (1.0 / sim_array) si el usuario lo solicitó, mejorando la legibilidad del reporte.

4) Se generan dos hojas por cada moneda:

- DF_Prom_MONEDA: Muestra el factor de descuento promedio esperado para cada tenor de la curva y en cada paso de tiempo futuro.

- Tasa_Prom_MONEDA: Convierte los factores de descuento promedio de vuelta a tasas de interés cero equivalentes. Esto a menudo es más intuitivo para un analista. 

- La fórmula r = -log(DF) / T se utiliza para esta conversión.

5) Se utiliza la funcionalidad de pd.ExcelWriter para crear un archivo Excel y escribir cada una de las tablas generadas en los pasos anteriores en una hoja de cálculo separada y con un nombre descriptivo. Esto consolida toda la información relevante en un único archivo de salida, fácil de compartir y analizar.

In [ ]:
# ==============================================================================
# Script Principal
# ==============================================================================

if __name__ == "__main__":
    try:
        # (1) --- PASOS INICIALES ---
        moneda_valoracion = input("¿Moneda de valoración? (ej: CLP, USD): ").strip().upper()
        t0_str = input("Fecha de valoración (YYYY-MM-DD o DD-MM-YYYY, default: hoy): ").strip()
        t0 = pd.to_datetime(t0_str, dayfirst=True) if t0_str else pd.to_datetime(datetime.now().date())
        print(f"Fecha de valoración establecida en: {t0.strftime('%Y-%m-%d')}")

        tc_reporte_inverso = False
        if moneda_valoracion == 'USD': # O cualquier otra lógica que prefieras
            respuesta_tc = input(f"En el reporte, ¿cómo quieres ver el TC? (1: USD/CLP, 2: CLP/USD) [Default: 1]: ").strip()
            if respuesta_tc == '1' or respuesta_tc == '':
                tc_reporte_inverso = True
            print(f"  El reporte mostrará el TC en formato {'USD/Moneda' if tc_reporte_inverso else 'Moneda/USD'}.")

        root = tk.Tk(); root.withdraw()
        ruta_forwards = filedialog.askopenfilename(title="Selecciona el archivo con los forwards")
        forwards_df = cargar_y_validar_forwards(ruta_forwards)
        print("✅ Paso 2: Forwards cargados y validados.")
        display(forwards_df.head())

        monedas_unicas = sorted(set(forwards_df['Moneda_Flujo_Activo']) | set(forwards_df['Moneda_Flujo_Pasivo']))
        print(f"✅ Paso 3: Monedas únicas identificadas: {monedas_unicas}")

        ruta_insumos = filedialog.askopenfilename(title="Selecciona el archivo con insumos de mercado")
        curvas_descuento, tasas_historicas, datos_fx, monedas_relacionadas = cargar_y_organizar_insumos(
            ruta_insumos, monedas_unicas, moneda_valoracion, t0
        )
        
        # --- MODELADO Y SIMULACIÓN ---
        print(f"\n✅ Paso 7: Parámetros de simulación definidos (Sims={NUM_SIMULACIONES}, Pasos={NUM_PASOS}).")
        simulaciones_tasas, simulaciones_tc = calibrar_y_simular_riesgos(
            tasas_historicas, datos_fx, monedas_unicas, monedas_relacionadas, moneda_valoracion
        )
        
        # --- VALORACIÓN Y CÁLCULO DE XVA ---
        valores_mtm_array, dfs_simulados = valorar_portafolio_vectorizado(
            forwards_df, t0, moneda_valoracion, curvas_descuento, simulaciones_tasas, simulaciones_tc
        )

        print("\n" + "="*50)
        print("VERIFICACIÓN: VALOR MtM INICIAL (t=0)")
        print("="*50)
        mtm_inicial = valores_mtm_array[:, 0, 0]
        df_mtm_inicial = pd.DataFrame({'ID': forwards_df['ID'], 'MtM_Inicial_Modelo': mtm_inicial})
        print("Compara estos valores con tu hoja de cálculo:")
        display(df_mtm_inicial.set_index('ID'))
        
        df_xva_results, df_ee_total = calcular_xva_con_neteo(
            forwards_df, valores_mtm_array, curvas_descuento, moneda_valoracion
        )
        
        # --- REPORTES FINALES ---
        print("\n" + "="*50)
        print("PASO 18: RESUMEN DE RESULTADOS (BVA = CVA - DVA)")
        print("="*50)
        display(df_xva_results.round(2))
        print(f"\n💼 CVA Total: {df_xva_results['CVA'].sum():,.2f} {moneda_valoracion}")
        print(f"💼 DVA Total: {df_xva_results['DVA'].sum():,.2f} {moneda_valoracion}")
        print(f"💼 BVA Total: {df_xva_results['BVA'].sum():,.2f} {moneda_valoracion}")

        ## <-- AJUSTE TC REPORTE: Pasar la preferencia a la función de reporte
        nombre_archivo_salida = "reporte_CVA_DVA_BVA_DETALLADO.xlsx"
        generar_reporte_detallado(
            ruta_salida=nombre_archivo_salida,
            forwards_df=forwards_df,
            df_xva=df_xva_results,
            df_ee=df_ee_total,
            valores_mtm=valores_mtm_array,
            simulaciones_tc=simulaciones_tc,
            dfs_simulados=dfs_simulados,
            curvas_descuento=curvas_descuento,
            moneda_valoracion=moneda_valoracion,
            t0=t0,
            ruta_forwards=ruta_forwards,
            ruta_insumos=ruta_insumos,
            tc_reporte_inverso=tc_reporte_inverso
        )
    
    except Exception as e:
        print("\n" + "!"*50)
        print(" OCURRIÓ UN ERROR INESPERADO ".center(50, "!"))
        print("!"*50)
        print(f"\nError: {e}")
        import traceback
        traceback.print_exc()

--> Esta sección es el punto de entrada y el flujo de ejecución principal del programa. No define funciones nuevas, sino que ejecuta la llamada a todas las funciones definidas anteriormente en una secuencia lógica y coherente, desde la entrada de datos hasta la generación del reporte final.

--> Código: 

# La línea if __name__ == "__main__": es una convención estándar en Python. Asegura que el código dentro de este bloque solo se ejecute cuando el archivo se corre directamente, y no si es importado como un módulo por otro script.

# La estructura try...except es un mecanismo de manejo de errores. Si ocurre cualquier error inesperado en cualquier parte del proceso, el programa no se detendrá bruscamente, sino que saltará al bloque except, imprimirá un mensaje de error detallado y finalizará de manera controlada.

1) Esta fase recopila toda la información necesaria para la ejecución:

- Se solicita interactivamente la moneda de valoración, la fecha y la preferencia para el reporte de tipo de cambio.

- Se abren ventanas de diálogo para que el usuario seleccione los archivos Excel con el portafolio y los insumos de mercado.

- Se llama a las funciones cargar_y_validar_forwards y cargar_y_organizar_insumos para leer y preparar todos los datos necesarios.

- Se extraen las monedas únicas del portafolio, un dato clave para los siguientes pasos.

2) En esta fase, se invoca a la función principal de modelado. Esta única llamada encapsula toda la complejidad de la calibración de Vasicek, el cálculo de la matriz de Cholesky y la simulación de miles de trayectorias futuras para todos los factores de riesgo.

3) En esta parte se calculan los ajustes XVA:

- Se llama a valorar_portafolio_vectorizado, pasándole las simulaciones de mercado para obtener el "cubo" de valores MtM.

- Se realiza una comprobación inmediata del valor MtM en t=0, comparándolo con los valores esperados. Este es un paso de validación del modelo en tiempo real.

- Con las valoraciones validadas, se llama a calcular_xva_con_neteo para realizar el cálculo final de CVA, DVA y BVA, incluyendo el neteo por contraparte.

4) La fase final se dedica a comunicar los resultados.

- Se imprime un resumen de los resultados de CVA, DVA y BVA directamente en la consola, para una visión rápida.

- Se llama a la función generar_reporte_detallado, pasándole todos los resultados y datos intermedios relevantes para que los consolide en el archivo Excel final.